In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from graphslam.graph import Graph
from graphslam.vertex import Vertex
from graphslam.edge.edge_odometry import EdgeOdometry
import matplotlib.pyplot as plt

# load data

In [ ]:
class ImageLoader:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        self.image_files.sort()  # Ensure images are in order
        self.current_index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_index >= len(self.image_files):
            raise StopIteration
        
        image_path = os.path.join(self.folder_path, self.image_files[self.current_index])
        image = cv2.imread(image_path)
        self.current_index += 1
        return image

# Usage
folder_path='sample_data/Room_back_home-2024-12-29_19-50-26/'
folder_path_img = folder_path+'/Camera'
image_loader = ImageLoader(folder_path_img)

In [ ]:
gyro = pd.read_csv(folder_path+'Gyroscope.csv')
gyro['time_diff_s'] = gyro['time'].diff().fillna(0) / 1e9
gyro['roll'] = (gyro['x']*gyro['time_diff_s']).cumsum()
gyro['pitch'] = (gyro['y']*gyro['time_diff_s']).cumsum()
gyro['yaw'] = (gyro['z']*gyro['time_diff_s']).cumsum()
gyro.drop(['x','y','z'],axis=1,inplace=True)

acc = pd.read_csv(folder_path+'/Accelerometer.csv')

motion = pd.merge(gyro, acc, on=['time','seconds_elapsed'],how='inner')
motion.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)
motion

In [ ]:
#1731164816467        in ms
#1731164816433211400  in ns
image_filenames = sorted(os.listdir(folder_path_img))[1:-1]
image_timestamps = [float(filename.split('.')[0]) for filename in image_filenames] 

def synchronize_data(imu_data, image_timestamps):
    synchronized_data = []
    for img_time in image_timestamps:
        # Find closest IMU timestamp
        closest_imu_index = (np.abs(imu_data['time'] - img_time*1e6)).argmin()
        synchronized_data.append({
            'image_time': img_time,
            'imu_roll': imu_data.iloc[closest_imu_index]['roll'],
            'imu_pitch': imu_data.iloc[closest_imu_index]['pitch'],
            'imu_yaw': imu_data.iloc[closest_imu_index]['yaw'],
            'imu_ax': imu_data.iloc[closest_imu_index]['ax'],
            'imu_ay': imu_data.iloc[closest_imu_index]['ay'],
            'imu_az': imu_data.iloc[closest_imu_index]['az'],
            'image_path': os.path.join(folder_path_img, image_filenames[image_timestamps.index(img_time)])
        })
    return pd.DataFrame(synchronized_data)

synchronized_df = synchronize_data(motion, image_timestamps)
synchronized_df['time_diff_s'] = synchronized_df['image_time'].diff().fillna(0) / 1e3


In [ ]:
synchronized_df.head()

In [ ]:
def extract_features(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    return keypoints, descriptors

# define slam

In [ ]:
# Pose Estimation with IMU Integration
def compute_pose(prev_keypoints, prev_descriptors, curr_keypoints, curr_descriptors, i):
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(prev_descriptors, curr_descriptors)

    # Filter matches to get only good matches
    good_matches = [m for m in matches if m.distance < 50]  # Adjust threshold as necessary

    if len(good_matches) < 5:
        print("Not enough matches found.",i)
        return None, None

    src_pts = np.float32([prev_keypoints[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([curr_keypoints[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Compute essential matrix and recover pose
    E, mask = cv2.findEssentialMat(src_pts, dst_pts)
    
    if E is None or E.shape != (3, 3):
        print("Essential matrix could not be computed.")
        print(E)
        return None, None

    # Use only inliers for recoverPose
    try:
        _, R, t, mask_pose = cv2.recoverPose(E, src_pts, dst_pts)
    except cv2.error as e:
        print(i)
        print(good_matches)
    

    return R, t

In [ ]:
# Initialize variables
trajectory = []
landmarks = []
prev_keypoints = None
prev_descriptors = None
prev_pose = np.eye(4)  # Initial pose (identity matrix)

for index in range(len(synchronized_df) - 1):
    curr_image_path = synchronized_df.iloc[index + 1]['image_path']
    
    # Extract features from current and previous images
    curr_keypoints, curr_descriptors = extract_features(curr_image_path)
    
    if prev_keypoints is not None:
        # Estimate motion using visual odometry and IMU data
        R, t = compute_pose(prev_keypoints, prev_descriptors, curr_keypoints, curr_descriptors, index)
        
        if R is not None and t is not None:
            # Update trajectory using IMU data for better accuracy
            imu_roll = synchronized_df.iloc[index + 1]['imu_roll']
            imu_pitch = synchronized_df.iloc[index + 1]['imu_pitch']
            imu_yaw = synchronized_df.iloc[index + 1]['imu_yaw']

            # Convert IMU angles to rotation matrix (assuming small angles)
            imu_rotation_matrix = cv2.Rodrigues(np.array([np.radians(imu_roll), np.radians(imu_pitch), np.radians(imu_yaw)]))[0]

            # Combine visual odometry with IMU data (simple integration)
            combined_rotation_matrix = imu_rotation_matrix @ R
            combined_translation_vector = t.flatten() + prev_pose[:3, :3] @ np.array([synchronized_df.iloc[index + 1]['imu_ax'], 
                                                                                       synchronized_df.iloc[index + 1]['imu_ay'], 
                                                                                       synchronized_df.iloc[index + 1]['imu_az']])

            # Update pose
            current_pose = np.eye(4)
            current_pose[:3, :3] = combined_rotation_matrix
            current_pose[:3, 3] = combined_translation_vector
            
            trajectory.append(current_pose[:3, 3])  # Store position

            # Add landmarks (e.g., if you detect new features)
            for kp in curr_keypoints:
                landmarks.append((kp.pt[0], kp.pt[1]))  # Store landmark positions

    # Update previous keypoints and descriptors for next iteration
    prev_keypoints = curr_keypoints
    prev_descriptors = curr_descriptors

# Convert trajectory to numpy array for visualization
trajectory = np.array(trajectory)

# Visualization of Trajectory and Landmarks
plt.figure(figsize=(10, 6))
plt.plot(trajectory[:, 0], trajectory[:, 1], label='Camera Trajectory')
plt.scatter(*zip(*landmarks), color='red', s=5)  # Plot landmarks in red
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Camera Trajectory with Landmarks')
plt.legend()
plt.show()

# reconstruct trajectory

In [ ]:
# Step 4: Pose Estimation from IMU Data
def compute_pose_from_imu(roll, pitch, yaw, ax,ay,az, dt):
    # Create rotation matrix from Euler angles
    R_x = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])
    
    R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])
    
    R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])

    # Combined rotation matrix
    R = R_z @ R_y @ R_x

    # Integrate acceleration to get velocity
    velocity_change = np.array([ax * dt, ay * dt, az * dt])
    
    # Transform velocity change into global frame using rotation matrix
    global_velocity_change = R @ velocity_change
    
    return R, global_velocity_change
    

In [ ]:
# Step 5: Initialize GraphSLAM and Variables
from graphslam.vertex import Vertex
from graphslam.edge import Edge

edges = []
vertices = []
g = Graph(edges, vertices)  # Create a new graph instance
landmarks = {}
trajectory = []
dt = 0.1  # Example time step (assuming constant time between frames)
velocity = np.zeros(3)  # Initialize velocity

for index in range(len(synchronized_df)):
    roll = synchronized_df.iloc[index]['imu_roll']
    pitch = synchronized_df.iloc[index]['imu_pitch']
    yaw = synchronized_df.iloc[index]['imu_yaw']
    
    # Get IMU acceleration data
    ax = synchronized_df.iloc[index]['imu_ax']
    ay = synchronized_df.iloc[index]['imu_ay']
    az = synchronized_df.iloc[index]['imu_az']

    # Compute pose from IMU data
    R, velocity_change = compute_pose_from_imu(roll, pitch, yaw, ax, ay, az, dt)
    
    # Update velocity based on acceleration (integrate)
    velocity += velocity_change
    
    if index == 0:
        current_position = np.zeros(3)  # Initial position at origin
    else:
        current_position += velocity * dt  # Integrate velocity to get position

    # Create vertex for current pose
    vertex = Vertex(index, current_position)
    vertices.append(vertex)
    
    # Create edge between consecutive poses
    if index > 0:
        edge = Edge(vertices[index-1], vertex, velocity_change * dt)
        edges.append(edge)

    trajectory.append(current_position)

    # Extract features from the current image
    curr_image_path = synchronized_df.iloc[index]['image_path']
    img = cv2.imread(curr_image_path)
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create()
    
    keypoints, descriptors = orb.detectAndCompute(gray_img, None)

    # Create landmarks based on detected keypoints (for simplicity)
    for kp in keypoints:
        landmark_id = len(landmarks) + len(synchronized_df) + 1
        
        if landmark_id not in landmarks:
            landmarks[landmark_id] = (kp.pt[0], kp.pt[1], current_position)  # Store landmark with its position
            
            # Add landmark as a vertex in the optimizer (assuming Z is height or depth estimation)
            g.add_landmark(landmark_id,
                           np.array([kp.pt[0], kp.pt[1], current_position[2]])) 

            # Add edge between current pose and landmark (assuming we have observations)
            g.add_edge(index, landmark_id)  # Edge from pose to landmark

# Optimize the graph after processing all frames
g.optimize()  # Perform optimization

# Convert trajectory to numpy array for visualization
trajectory = np.array(trajectory)

In [ ]:
trajectory[:,0]

In [ ]:
import plotly.graph_objs as go
# Create a scatter plot for the trajectory
fig = go.Figure(data=[go.Scatter3d(
    x=trajectory[:,0],
    y=trajectory[:,1],
    z=trajectory[:,2],
    mode='lines',
    line=dict(color='blue', width=2),
)])

fig.update_layout(
    title='3D Trajectory Plot with Grid',
    scene=dict(
        xaxis=dict(title='X Position', gridcolor='lightgray', showgrid=True),
        yaxis=dict(title='Y Position', gridcolor='lightgray', showgrid=True),
        zaxis=dict(title='Z Position', gridcolor='lightgray', showgrid=True),
        aspectmode='cube'  # Optional: Keep the aspect ratio equal
    ),
    margin=dict(l=0, r=0, b=0, t=40)  # Adjust margins for better visibility
)

# Show the interactive plot
fig.show()

# with optimization